## 基于多目标GP的符号回归

多目标GP是指使用多个目标函数来评估GP树的适应度。在符号回归问题中，通常使用均方误差（MSE）作为目标函数。然而，MSE并不能很好地反映模型的复杂度，因此，我们还可以使用树的大小作为目标函数。这样，就可以得到更为精简的模型。

In [5]:
import math
import operator
import random
from deap import base, creator, tools, gp, algorithms

# 定义评估函数，包含两个目标：均方误差和树的大小
def evalSymbReg(individual,pset):
    # 编译GP树为函数
    func = gp.compile(expr=individual, pset=pset)
    # 计算均方误差（Mean Square Error，MSE）
    mse = ((func(x) - x**2)**2 for x in range(-10, 10))
    # 计算GP树的大小
    size = len(individual)
    return math.fsum(mse), size

# 修改适应度函数，包含两个权重：MSE和树的大小。MSE是最小化，树的大小也是最小化
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMulti)

### 遗传算子
遗传算子基本不需要修改。由于是多目标优化问题，所以选择算子需要使用NSGA2（Non-dominated Sorting Genetic Algorithm II）。
NSGA2算法的基本思想是，首先对种群中的个体进行非支配排序，然后根据非支配排序的结果计算拥挤度距离，最后根据非支配排序和拥挤度距离两个指标对个体进行排序。

In [6]:
import random

# 定义函数集合和终端集合
pset = gp.PrimitiveSet("MAIN", arity=1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(operator.neg, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1, 1))
pset.renameArguments(ARG0='x')

# 定义遗传编程操作
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evalSymbReg, pset=pset)
toolbox.register("select", tools.selNSGA2)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)

### 算法模块
DEAP算法包提供了eaMuPlusLambda函数，可以比较方便地使用NSGA2的环境选择算子。  
理想情况下，最好还是自行实现演化函数，这样才能完整地使用NSGA-II算法中的锦标赛选择算子。  
NSGA-II算法中的锦标赛选择算子是指，首先从种群中随机选择两个个体，然后根据非支配排序和拥挤度距离两个指标对两个个体进行排序，最后选择排名较高的个体作为父代。简单起见，我们忽略了锦标赛选择算子。

In [7]:
import numpy
from deap import algorithms

# 统计指标
stats_fit = tools.Statistics(lambda ind: ind.fitness.values[0])
stats_size = tools.Statistics(lambda ind: ind.fitness.values[1])
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)

population = toolbox.population(n=50)
pop, log  = algorithms.eaMuPlusLambda(population=population,
                           toolbox=toolbox, mu=len(population),lambda_=len(population),
                                      cxpb=0.9, mutpb=0.1, ngen=10, stats=mstats, halloffame=None, verbose=True)

# 最佳个体
best_ind = tools.selBest(pop, 1)[0]
print('Best individual is:\n', best_ind)
print('\nWith fitness:', best_ind.fitness.values)


基于优化结果，我们还可以绘制Pareto前沿，以便于选择最终的模型。

In [8]:
from matplotlib import pyplot as plt
import seaborn as sns

# 非支配排序
fronts = tools.sortNondominated(pop, len(pop), first_front_only=True)

# Pareto前沿
pareto_front = fronts[0]
fitnesses = [ind.fitness.values for ind in pareto_front]

# 分离均方误差和树的大小
mse = [fit[0] for fit in fitnesses]
sizes = [fit[1] for fit in fitnesses]

# 使用seaborn绘制散点图
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.scatterplot(x=mse, y=sizes, palette="viridis", s=60, edgecolor="w", alpha=0.7)
plt.xlabel('Mean Square Error')
plt.ylabel('Size of the GP Tree')
plt.title('Pareto Front')
plt.show()